In [114]:
import pandas as pd

# 데이터

## 1. 침수지역

In [115]:
침수지역=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\침수지역2022.csv")
침수횟수=pd.DataFrame(침수지역[["GU_NAME","ADM_NM"]].value_counts()).reset_index()

In [116]:
침수횟수

,GU_NAME,ADM_NM,count
0,영등포구,대림2동,880
1,영등포구,대림1동,527
2,영등포구,신길6동,397
3,영등포구,신길5동,362
4,동작구,사당1동,270
...,...,...,...
272,은평구,신사2동,1
273,은평구,응암2동,1
274,노원구,월계2동,1
275,종로구,가회동,1


## 2. 강우량

In [117]:
강우량=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\강우량.xlsx")
구단위=pd.DataFrame(강우량.groupby(["구청명","자료수집 시각"])["10분우량"].mean())
구단위.reset_index(inplace=True)
#구단위
구단위총합=구단위.groupby(['구청명'])['10분우량'].agg(["sum","count"]).reset_index()
구단위총합.rename(columns={"sum":'총우량',"count":'측정횟수(10분단위)'},inplace=True)
구단위총합["면적당시우량"]=구단위총합["총우량"]/구단위총합["측정횟수(10분단위)"]*6
구단위총합
구단위총합
인구밀도=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\서울시_행정동별_인구밀도_2023.csv")
인구밀도
구비율_행정동면적=pd.DataFrame(인구밀도.groupby(["구"])["면적 (k㎡)"].sum()).reset_index()
구비율_행정동면적=pd.merge(인구밀도,구비율_행정동면적,left_on="구",right_on='구')
구비율_행정동면적.rename(columns={'면적 (k㎡)_x':'동면적(k㎡)',"면적 (k㎡)_y":"구면적(k㎡)"},inplace=True)
구비율_행정동면적['동/구면적']=구비율_행정동면적["동면적(k㎡)"]/구비율_행정동면적['구면적(k㎡)']
merge_data=pd.merge(구비율_행정동면적,구단위총합,left_on="구",right_on="구청명")
merge_data['동단위우량']=merge_data["면적당시우량"]*merge_data["동면적(k㎡)"]
merge_data["동단위시우량"]=merge_data["면적당시우량"]*merge_data["동/구면적"]
merge_data.drop(["index",'행정구역코드','구면적(k㎡)','동/구면적','구청명','총우량','측정횟수(10분단위)','면적당시우량'],axis=1,inplace=True)
merge_data.rename(columns={"계 (명)":'인구수'},inplace=True)
merge_data.rename(columns={"구":"GU_NAME","행정동":'ADM_NM'},inplace=True)
merge_data
merge_data=pd.merge(merge_data,침수횟수,how="left").rename(columns={'count':"침수횟수"})

In [118]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GU_NAME      426 non-null    object 
 1   ADM_NM       426 non-null    object 
 2   동면적(k㎡)      426 non-null    float64
 3   인구수          426 non-null    int64  
 4   인구밀도(명/km²)  426 non-null    float64
 5   동단위우량        426 non-null    float64
 6   동단위시우량       426 non-null    float64
 7   침수횟수         272 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 26.8+ KB


In [119]:
merge_data.fillna(0,inplace=True) #침수횟수 nan->0

In [120]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GU_NAME      426 non-null    object 
 1   ADM_NM       426 non-null    object 
 2   동면적(k㎡)      426 non-null    float64
 3   인구수          426 non-null    int64  
 4   인구밀도(명/km²)  426 non-null    float64
 5   동단위우량        426 non-null    float64
 6   동단위시우량       426 non-null    float64
 7   침수횟수         426 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 26.8+ KB


## 3. 경사도

In [121]:
경사도=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\decisiontree\경사도.xlsx")
경사도=pd.DataFrame(경사도.groupby(["GU_NAME","ADM_NM"])["경사도"].mean()).reset_index()

In [122]:
merge_data=pd.merge(merge_data,경사도,how='inner')
#merge_data["경사도"].fillna(0,inplace=True)

In [123]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GU_NAME      415 non-null    object 
 1   ADM_NM       415 non-null    object 
 2   동면적(k㎡)      415 non-null    float64
 3   인구수          415 non-null    int64  
 4   인구밀도(명/km²)  415 non-null    float64
 5   동단위우량        415 non-null    float64
 6   동단위시우량       415 non-null    float64
 7   침수횟수         415 non-null    float64
 8   경사도          415 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 29.3+ KB


### 4. 배수등급양호비율

In [124]:
배수=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\배수등급양호비율.csv")
배수

,GU_NAME,ADM_NM,양호개수,배수전체개수,양호비율
0,송파구,가락1동,157,240,0.654167
1,송파구,가락2동,96,114,0.842105
2,송파구,가락본동,85,125,0.680000
3,구로구,가리봉동,75,109,0.688073
4,금천구,가산동,274,449,0.610245
...,...,...,...,...,...
412,용산구,효창동,102,104,0.980769
413,용산구,후암동,178,178,1.000000
414,동대문구,휘경1동,28,69,0.405797
415,동대문구,휘경2동,52,173,0.300578


In [125]:
merge_data=pd.merge(merge_data,배수[["GU_NAME","ADM_NM","양호비율"]],how="inner")
#merge_data.fillna(배수["양호비율"].mean(),inplace=True)

In [126]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GU_NAME      408 non-null    object 
 1   ADM_NM       408 non-null    object 
 2   동면적(k㎡)      408 non-null    float64
 3   인구수          408 non-null    int64  
 4   인구밀도(명/km²)  408 non-null    float64
 5   동단위우량        408 non-null    float64
 6   동단위시우량       408 non-null    float64
 7   침수횟수         408 non-null    float64
 8   경사도          408 non-null    float64
 9   양호비율         408 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 32.0+ KB


### 5. 녹지면적

In [127]:
녹지=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\행정동별 녹지면적.csv")

In [128]:
merge_data=pd.merge(merge_data,녹지,how='inner')
merge_data[merge_data["GREEN_AREA"].isna()]

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,GREEN_AREA


In [129]:
merge_data.rename(columns={"GREEN_AREA":'녹지면적'},inplace=True)

### 5.하천까지의 거리

In [130]:
하천=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\하천까지의_거리.csv")
하천['하천거리(km)']=하천["HubDist"]/1000

In [131]:
merge_data=pd.merge(merge_data,하천[["GU_NAME","ADM_NM",'하천거리(km)']],how='inner')

In [132]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GU_NAME      408 non-null    object 
 1   ADM_NM       408 non-null    object 
 2   동면적(k㎡)      408 non-null    float64
 3   인구수          408 non-null    int64  
 4   인구밀도(명/km²)  408 non-null    float64
 5   동단위우량        408 non-null    float64
 6   동단위시우량       408 non-null    float64
 7   침수횟수         408 non-null    float64
 8   경사도          408 non-null    float64
 9   양호비율         408 non-null    float64
 10  녹지면적         408 non-null    float64
 11  하천거리(km)     408 non-null    float64
dtypes: float64(9), int64(1), object(2)
memory usage: 38.4+ KB


### 6. 빗물펌프장

In [133]:
빗물펌프장_거리=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\거리계산.xlsx")
빗물펌프장_거리.rename(columns=dict(zip(빗물펌프장_거리.columns,["ADM_NM","펌프장","빗물펌프장까지의거리(m)"])),inplace=True)
빗물펌프장_정보=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\빗물펌프장_전처리.xlsx")
펌프장거리정보=pd.merge(빗물펌프장_거리,빗물펌프장_정보)
펌프장거리정보.drop(["Unnamed: 0","위치"],axis=1,inplace=True)
펌프장=pd.DataFrame(펌프장거리정보.groupby(["GU_NAME","ADM_NM"])["빗물펌프장까지의거리(m)"].min()).reset_index()
merge_data=pd.merge(merge_data,펌프장,how='inner')

In [134]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GU_NAME        351 non-null    object 
 1   ADM_NM         351 non-null    object 
 2   동면적(k㎡)        351 non-null    float64
 3   인구수            351 non-null    int64  
 4   인구밀도(명/km²)    351 non-null    float64
 5   동단위우량          351 non-null    float64
 6   동단위시우량         351 non-null    float64
 7   침수횟수           351 non-null    float64
 8   경사도            351 non-null    float64
 9   양호비율           351 non-null    float64
 10  녹지면적           351 non-null    float64
 11  하천거리(km)       351 non-null    float64
 12  빗물펌프장까지의거리(m)  351 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 35.8+ KB


### 7. 제방암거

In [135]:
제방=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\제방암거수문개수.csv",encoding="euc-kr")
제방
merge_data=pd.merge(merge_data,제방,how='inner')
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GU_NAME        350 non-null    object 
 1   ADM_NM         350 non-null    object 
 2   동면적(k㎡)        350 non-null    float64
 3   인구수            350 non-null    int64  
 4   인구밀도(명/km²)    350 non-null    float64
 5   동단위우량          350 non-null    float64
 6   동단위시우량         350 non-null    float64
 7   침수횟수           350 non-null    float64
 8   경사도            350 non-null    float64
 9   양호비율           350 non-null    float64
 10  녹지면적           350 non-null    float64
 11  하천거리(km)       350 non-null    float64
 12  빗물펌프장까지의거리(m)  350 non-null    float64
 13  제방             350 non-null    int64  
 14  수문             350 non-null    int64  
 15  암거             350 non-null    int64  
dtypes: float64(10), int64(4), object(2)
memory usage: 43.9+ KB


In [136]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,녹지면적,하천거리(km),빗물펌프장까지의거리(m),제방,수문,암거
0,용산구,후암동,0.86,16740,19465.116280,6.705766,0.306760,0.0,0.396250,1.000000,283.456489,0.346532,1819.097470,2,0,7
1,용산구,용산2가동,1.96,10316,5263.265306,15.282909,0.699127,0.0,0.319000,0.600719,275.763890,1.565791,1067.096748,1,3,15
2,용산구,남영동,1.19,7552,6346.218487,9.278909,0.424470,2.0,0.115000,0.367232,7.200488,1.148920,1068.749272,0,0,23
3,용산구,원효로2동,0.71,13779,19407.042250,5.536156,0.253255,1.0,0.123000,0.902299,74.757260,0.522160,328.424579,1,0,22
4,용산구,효창동,0.44,10902,24777.272730,3.430857,0.156947,0.0,0.126667,0.980769,136.136473,3.363537,1284.697842,1,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,4.093324,1.340566,1748.827684,0,0,17
346,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,94.996411,1.053767,1960.757507,4,0,21
347,강동구,둔촌1동,0.92,74,80.434783,8.092320,0.329090,3.0,0.151786,0.738636,41.522737,1.112581,2558.601645,0,0,9
348,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,229.326026,0.286658,2376.349662,19,1,68


### 9. 도로건물면적

In [137]:
도로건물면적=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\도로_건물.csv")

In [138]:
도로건물면적[["ADM_NM","도로_건물_면적"]]

,ADM_NM,도로_건물_면적
0,가락1동,2.881325e+06
1,가락2동,1.307882e+06
2,가락본동,2.458442e+06
3,가리봉동,3.107159e+06
4,가산동,1.991458e+07
...,...,...
418,효창동,9.139857e+05
419,후암동,1.123626e+06
420,휘경1동,6.180668e+05
421,휘경2동,8.611521e+05


In [139]:
도로건물면적[도로건물면적["ADM_NM"]=="사직동"]

,ADM_NM,ROAD_BT,ROAD_LT,ROAD_AREA,SHAPE_AREA,도로_건물_면적
161,사직동,1523.0,57391.0,1136830.0,306229.52,1443059.52


In [140]:
merge_data=pd.merge(merge_data,도로건물면적[["GU_NAME","ADM_NM","도로_건물_면적"]],how='inner')

KeyError: "['GU_NAME'] not in index"

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,녹지면적,하천거리(km),빗물펌프장까지의거리(m),제방,수문,암거,도로_건물_면적
0,용산구,후암동,0.86,16740,19465.116280,6.705766,0.306760,0.0,0.396250,1.000000,283.456489,0.346532,1819.097470,2,0,7,1.123626e+06
1,용산구,용산2가동,1.96,10316,5263.265306,15.282909,0.699127,0.0,0.319000,0.600719,275.763890,1.565791,1067.096748,1,3,15,7.881864e+05
2,용산구,남영동,1.19,7552,6346.218487,9.278909,0.424470,2.0,0.115000,0.367232,7.200488,1.148920,1068.749272,0,0,23,1.034960e+06
3,용산구,원효로2동,0.71,13779,19407.042250,5.536156,0.253255,1.0,0.123000,0.902299,74.757260,0.522160,328.424579,1,0,22,3.490083e+06
4,용산구,효창동,0.44,10902,24777.272730,3.430857,0.156947,0.0,0.126667,0.980769,136.136473,3.363537,1284.697842,1,0,20,9.139857e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,4.093324,1.340566,1748.827684,0,0,17,1.865436e+06
346,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,94.996411,1.053767,1960.757507,4,0,21,3.340017e+06
347,강동구,둔촌1동,0.92,74,80.434783,8.092320,0.329090,3.0,0.151786,0.738636,41.522737,1.112581,2558.601645,0,0,9,3.201373e+06
348,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,229.326026,0.286658,2376.349662,19,1,68,2.868552e+06


### 10.관공서

In [ ]:
관공서=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\인프라_행정동별_관공서합계(경찰서,병원,소방서).xlsx")
merge_data=pd.merge(merge_data,관공서.rename(columns={'동별':"ADM_NM"})[["ADM_NM","병원","경찰","소방서"]],how='inner')

In [ ]:
merge_data["ADM_NM"].value_counts()

ADM_NM
신사동     8
도림동     1
사당3동    1
사당1동    1
상도4동    1
       ..
상계8동    1
상계5동    1
상계2동    1
상계1동    1
강일동     1
Name: count, Length: 349, dtype: int64

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,녹지면적,하천거리(km),빗물펌프장까지의거리(m),제방,수문,암거,도로_건물_면적,병원,경찰,소방서
0,용산구,후암동,0.86,16740,19465.116280,6.705766,0.306760,0.0,0.396250,1.000000,283.456489,0.346532,1819.097470,2,0,7,1.123626e+06,15,1,1
1,용산구,용산2가동,1.96,10316,5263.265306,15.282909,0.699127,0.0,0.319000,0.600719,275.763890,1.565791,1067.096748,1,3,15,7.881864e+05,6,0,0
2,용산구,남영동,1.19,7552,6346.218487,9.278909,0.424470,2.0,0.115000,0.367232,7.200488,1.148920,1068.749272,0,0,23,1.034960e+06,36,0,0
3,용산구,원효로2동,0.71,13779,19407.042250,5.536156,0.253255,1.0,0.123000,0.902299,74.757260,0.522160,328.424579,1,0,22,3.490083e+06,7,0,0
4,용산구,효창동,0.44,10902,24777.272730,3.430857,0.156947,0.0,0.126667,0.980769,136.136473,3.363537,1284.697842,1,0,20,9.139857e+05,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,4.093324,1.340566,1748.827684,0,0,17,1.865436e+06,80,0,0
347,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,94.996411,1.053767,1960.757507,4,0,21,3.340017e+06,103,0,1
348,강동구,둔촌1동,0.92,74,80.434783,8.092320,0.329090,3.0,0.151786,0.738636,41.522737,1.112581,2558.601645,0,0,9,3.201373e+06,1,0,0
349,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,229.326026,0.286658,2376.349662,19,1,68,2.868552e+06,28,2,0


### 11. 노후건물

In [ ]:
노후건물=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\개수지롱.csv")
merge_data=pd.merge(merge_data,노후건물[["GU_NAME","ADM_NM","NUMPOINTS"]].rename(columns={'NUMPOINTS':"노후건물개수"}),how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,...,하천거리(km),빗물펌프장까지의거리(m),제방,수문,암거,도로_건물_면적,병원,경찰,소방서,노후건물개수
0,용산구,후암동,0.86,16740,19465.116280,6.705766,0.306760,0.0,0.396250,1.000000,...,0.346532,1819.097470,2,0,7,1.123626e+06,15,1,1,1177
1,용산구,용산2가동,1.96,10316,5263.265306,15.282909,0.699127,0.0,0.319000,0.600719,...,1.565791,1067.096748,1,3,15,7.881864e+05,6,0,0,935
2,용산구,남영동,1.19,7552,6346.218487,9.278909,0.424470,2.0,0.115000,0.367232,...,1.148920,1068.749272,0,0,23,1.034960e+06,36,0,0,831
3,용산구,원효로2동,0.71,13779,19407.042250,5.536156,0.253255,1.0,0.123000,0.902299,...,0.522160,328.424579,1,0,22,3.490083e+06,7,0,0,620
4,용산구,효창동,0.44,10902,24777.272730,3.430857,0.156947,0.0,0.126667,0.980769,...,3.363537,1284.697842,1,0,20,9.139857e+05,12,0,0,419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,...,1.340566,1748.827684,0,0,17,1.865436e+06,80,0,0,425
349,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,...,1.053767,1960.757507,4,0,21,3.340017e+06,103,0,1,892
350,강동구,둔촌1동,0.92,74,80.434783,8.092320,0.329090,3.0,0.151786,0.738636,...,1.112581,2558.601645,0,0,9,3.201373e+06,1,0,0,11
351,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,...,0.286658,2376.349662,19,1,68,2.868552e+06,28,2,0,574


### 12. 소득가구

In [ ]:
# 소득가구=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\decisiontree\반출_행정동_소득가구_수정2차.xlsx")
# merge_data=pd.merge(merge_data,소득가구.rename(columns={"읍면동":'ADM_NM'})[["ADM_NM",'1분위소득가구비율(%)', '2분위소득가구비율(%)', '3분위소득가구비율(%)',
#        '4분위소득가구비율(%)', '5분위소득가구비율(%)']],how='inner')

0       사직동
1       삼청동
2       부암동
3       평창동
4       무악동
       ... 
418    성내3동
419    둔촌2동
420    암사1동
421    천호2동
422      길동
Name: 읍면동, Length: 423, dtype: object

### 13. 월평균 소득

In [ ]:
소득소비=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\서울시 상권분석서비스(소득소비-행정동).csv",encoding='euc-kr')
소득소비.rename(columns={'행정동_코드_명':'ADM_NM'},inplace=True)
# 월평균소득,지출은 2022년 평균으로 구함
i1=소득소비["기준_년분기_코드"]==20221
i2=소득소비["기준_년분기_코드"]==20222
i3=소득소비["기준_년분기_코드"]==20223
i4=소득소비["기준_년분기_코드"]==20224
temp=인구밀도.dropna()
temp.rename(columns={"행정구역코드":"행정동코드","행정동":'ADM_NM',"구":"GU_NAME"})
소득소비=pd.merge(소득소비,temp.rename(columns={"행정구역코드":"행정동코드","행정동":'ADM_NM',"구":"GU_NAME"}))
소득소비=pd.DataFrame(소득소비[i1|i2|i3|i4].groupby(["GU_NAME","ADM_NM"])[["월_평균_소득_금액",'지출_총금액']].sum()).reset_index()
소득소비[["월_평균_소득_금액"]]=소득소비[["월_평균_소득_금액"]].applymap(lambda x:x/4)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_12664\2410546092.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  소득소비=pd.DataFrame(소득소비[i1|i2|i3|i4].groupby(["GU_NAME","ADM_NM"])[["월_평균_소득_금액",'지출_총금액']].sum()).reset_index()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_12664\2410546092.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  소득소비[["월_평균_소득_금액"]]=소득소비[["월_평균_소득_금액"]].applymap(lambda x:x/4)


In [ ]:
merge_data=pd.merge(merge_data,소득소비,how='inner')

,GU_NAME,ADM_NM,월_평균_소득_금액,지출_총금액
0,강남구,개포2동,23153330.25,35370403000
1,강남구,논현1동,14920979.25,705624566000
2,강남구,대치4동,8202288.00,461972448000
3,강남구,삼성1동,23155194.75,1372221568000
4,강남구,역삼2동,24376871.25,11465112366000
...,...,...,...,...
83,종로구,평창동,20458217.25,44711860000
84,중구,광희동,14757581.25,203516353000
85,중구,다산동,12119944.50,54152172000
86,중랑구,망우본동,11069211.00,120839018000


In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,...,경찰,소방서,노후건물개수,1분위소득가구비율(%),2분위소득가구비율(%),3분위소득가구비율(%),4분위소득가구비율(%),5분위소득가구비율(%),월_평균_소득_금액,연간_총지출


### 15.노인 장애인 기초생활수급자

In [ ]:
노인=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\장애인노인비율.xlsx")

In [ ]:
xxx

NameError: name 'xxx' is not defined

In [ ]:
merge_data=pd.merge(merge_data,노인.rename(columns={"소계":"ADM_NM",27958:"노인수"}),how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,...,소방서,노후건물개수,1분위소득가구비율(%),2분위소득가구비율(%),3분위소득가구비율(%),4분위소득가구비율(%),5분위소득가구비율(%),월_평균_소득_금액,연간_총지출,노인수
0,종로구,사직동,1.23,9371,7618.699187,9.062665,0.379032,0.0,0.153750,0.245283,...,0,485,0.158391,0.142992,0.214959,0.180704,0.302640,5007768.0,223170702000,1808
1,종로구,삼청동,1.49,2616,1755.704698,10.978350,0.459153,0.0,0.250000,0.832402,...,0,344,0.240190,0.211653,0.229489,0.184304,0.133175,3442534.0,11857714000,626
2,종로구,부암동,2.27,9460,4167.400881,16.725406,0.699515,2.0,0.314583,0.906250,...,1,972,0.177386,0.169309,0.195334,0.212384,0.245588,3647449.0,6377388000,1814
3,종로구,평창동,8.87,17743,2000.338219,65.354340,2.733348,2.0,0.229615,0.974132,...,0,1331,0.119703,0.139261,0.173669,0.199565,0.367983,4504006.0,9726599000,3495
4,종로구,무악동,0.36,8045,22347.222220,2.652487,0.110936,0.0,0.346667,1.000000,...,0,101,0.157974,0.118292,0.160620,0.181028,0.381708,5136717.0,8331265000,1476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,강동구,성내2동,0.67,23414,34946.268660,5.893320,0.239663,19.0,0.055000,0.886957,...,0,1164,0.221124,0.224524,0.233450,0.202741,0.118160,2824592.0,28749704000,4469
464,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,...,0,425,0.180404,0.192774,0.234538,0.218861,0.173423,2897346.0,27366893000,4098
465,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,...,1,892,0.160445,0.201655,0.231001,0.214678,0.192221,2840603.0,48142577000,8478
466,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,...,0,574,0.137338,0.148917,0.193626,0.235011,0.285108,3414406.0,17302683000,4250


In [ ]:
장애인["동별"].unique()

array(['사직동', '삼청동', '부암동', '평창동', '무악동', '교남동', '가회동', '종로1.2.3.4가동',
       '종로5.6가동', '이화동', '창신1동', '창신2동', '창신3동', '숭인1동', '숭인2동', '청운효자동',
       '혜화동', '소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동', '신당5동',
       '황학동', '중림동', '신당동', '다산동', '약수동', '청구동', '동화동', '후암동', '용산2가동',
       '남영동', '원효로2동', '효창동', '용문동', '이촌1동', '이촌2동', '이태원1동', '이태원2동',
       '서빙고동', '보광동', '청파동', '원효로1동', '한강로동', '한남동', '왕십리2동', '마장동',
       '사근동', '행당1동', '행당2동', '응봉동', '금호1가동', '금호4가동', '성수1가1동', '성수1가2동',
       '성수2가1동', '성수2가3동', '송정동', '용답동', '왕십리도선동', '금호2.3가동', '옥수동',
       '화양동', '군자동', '중곡1동', '중곡2동', '중곡3동', '중곡4동', '능동', '구의1동', '구의2동',
       '구의3동', '광장동', '자양1동', '자양2동', '자양3동', '자양4동', '회기동', '휘경1동',
       '휘경2동', '청량리동', '용신동', '제기동', '전농1동', '전농2동', '답십리2동', '장안1동',
       '장안2동', '이문1동', '이문2동', '답십리1동', '면목2동', '면목4동', '면목5동', '면목7동',
       '상봉1동', '상봉2동', '중화1동', '중화2동', '묵1동', '묵2동', '망우3동', '신내1동',
       '신내2동', '면목본동', '면목3.8동', '망우본동', '돈암1동', '돈암2동', '안암동', '보문동',
   

In [ ]:
merge_data=pd.merge(merge_data,장애인.rename(columns={'동별':"ADM_NM"}),how='inner')
merge_data.rename(columns={'장애인 현황(장애유형별/동별)':'장애인수'},inplace=True)

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,...,노후건물개수,1분위소득가구비율(%),2분위소득가구비율(%),3분위소득가구비율(%),4분위소득가구비율(%),5분위소득가구비율(%),월_평균_소득_금액,연간_총지출,노인수,장애인수
0,종로구,사직동,1.23,9371,7618.699187,9.062665,0.379032,0.0,0.153750,0.245283,...,485,0.158391,0.142992,0.214959,0.180704,0.302640,5007768.0,223170702000,1808,277
1,종로구,삼청동,1.49,2616,1755.704698,10.978350,0.459153,0.0,0.250000,0.832402,...,344,0.240190,0.211653,0.229489,0.184304,0.133175,3442534.0,11857714000,626,108
2,종로구,부암동,2.27,9460,4167.400881,16.725406,0.699515,2.0,0.314583,0.906250,...,972,0.177386,0.169309,0.195334,0.212384,0.245588,3647449.0,6377388000,1814,318
3,종로구,평창동,8.87,17743,2000.338219,65.354340,2.733348,2.0,0.229615,0.974132,...,1331,0.119703,0.139261,0.173669,0.199565,0.367983,4504006.0,9726599000,3495,513
4,종로구,무악동,0.36,8045,22347.222220,2.652487,0.110936,0.0,0.346667,1.000000,...,101,0.157974,0.118292,0.160620,0.181028,0.381708,5136717.0,8331265000,1476,322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,강동구,성내2동,0.67,23414,34946.268660,5.893320,0.239663,19.0,0.055000,0.886957,...,1164,0.221124,0.224524,0.233450,0.202741,0.118160,2824592.0,28749704000,4469,1071
528,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,...,425,0.180404,0.192774,0.234538,0.218861,0.173423,2897346.0,27366893000,4098,876
529,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,...,892,0.160445,0.201655,0.231001,0.214678,0.192221,2840603.0,48142577000,8478,1983
530,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,...,574,0.137338,0.148917,0.193626,0.235011,0.285108,3414406.0,17302683000,4250,935


In [ ]:
수급자=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\수급자.xlsx")

In [ ]:
수급자[["소계",5559]]

,소계,5559
0,사직동,125
1,삼청동,45
2,부암동,174
3,평창동,144
4,무악동,206
...,...,...
470,천호2동,2002
471,길동,1637
472,상일1동,396
473,상일2동,689


In [ ]:
merge_data["ADM_NM"].unique()

array(['사직동', '삼청동', '부암동', '평창동', '무악동', '가회동', '이화동', '혜화동', '창신1동',
       '창신2동', '창신3동', '숭인1동', '숭인2동', '청운효자동', '소공동', '회현동', '명동', '필동',
       '장충동', '광희동', '을지로동', '신당5동', '황학동', '중림동', '다산동', '약수동', '청구동',
       '동화동', '후암동', '용산2가동', '남영동', '원효로2동', '효창동', '용문동', '이촌1동',
       '이촌2동', '이태원1동', '이태원2동', '서빙고동', '보광동', '청파동', '원효로1동', '한강로동',
       '한남동', '왕십리2동', '마장동', '사근동', '행당1동', '행당2동', '응봉동', '금호1가동',
       '금호4가동', '성수1가1동', '성수1가2동', '성수2가1동', '성수2가3동', '송정동', '용답동',
       '왕십리도선동', '옥수동', '화양동', '군자동', '중곡1동', '중곡3동', '중곡4동', '능동',
       '구의1동', '구의2동', '구의3동', '광장동', '자양1동', '자양2동', '자양3동', '자양4동',
       '회기동', '휘경1동', '휘경2동', '청량리동', '용신동', '제기동', '전농1동', '전농2동',
       '답십리1동', '답십리2동', '장안1동', '장안2동', '이문1동', '이문2동', '면목2동', '면목4동',
       '면목5동', '면목7동', '상봉1동', '상봉2동', '중화1동', '중화2동', '묵1동', '묵2동',
       '망우3동', '신내1동', '신내2동', '면목본동', '망우본동', '안암동', '보문동', '정릉1동',
       '정릉3동', '정릉4동', '길음1동', '길음2동', '월곡1동', '월곡2동', '장위1동', '장위3동',
       '성북동', '삼

In [ ]:
수급자["소계"].unique()

array(['사직동', '삼청동', '부암동', '평창동', '무악동', '교남동', '가회동', '종로1.2.3.4가동',
       '종로5.6가동', '이화동', '창신1동', '창신2동', '창신3동', '숭인1동', '숭인2동', '청운효자동',
       '혜화동', '기타', '소계', '소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동',
       '신당5동', '황학동', '중림동', '신당동', '다산동', '약수동', '청구동', '동화동', '후암동',
       '용산2가동', '남영동', '원효로2동', '효창동', '용문동', '이촌1동', '이촌2동', '이태원1동',
       '이태원2동', '서빙고동', '보광동', '청파동', '원효로1동', '한강로동', '한남동', '왕십리2동',
       '마장동', '사근동', '행당1동', '행당2동', '응봉동', '금호1가동', '금호4가동', '성수1가1동',
       '성수1가2동', '성수2가1동', '성수2가3동', '송정동', '용답동', '왕십리도선동', '금호2.3가동',
       '옥수동', '화양동', '군자동', '중곡1동', '중곡2동', '중곡3동', '중곡4동', '능동', '구의1동',
       '구의2동', '구의3동', '광장동', '자양1동', '자양2동', '자양3동', '자양4동', '회기동',
       '휘경1동', '휘경2동', '청량리동', '용신동', '제기동', '전농1동', '전농2동', '답십리2동',
       '장안1동', '장안2동', '이문1동', '이문2동', '답십리1동', '면목2동', '면목4동', '면목5동',
       '면목7동', '상봉1동', '상봉2동', '중화1동', '중화2동', '묵1동', '묵2동', '망우3동',
       '신내1동', '신내2동', '면목본동', '면목3.8동', '망우본동', '돈암1동', '돈암2동', '안암동'

In [ ]:
수급자.rename(columns={"소계":"ADM_NM",5559:"노인수"},inplace=True)
수급자

,ADM_NM,노인수
0,사직동,125
1,삼청동,45
2,부암동,174
3,평창동,144
4,무악동,206
...,...,...
470,천호2동,2002
471,길동,1637
472,상일1동,396
473,상일2동,689


In [ ]:
merge_data=pd.merge(merge_data,수급자, on= "ADM_NM")

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,...,1분위소득가구비율(%),2분위소득가구비율(%),3분위소득가구비율(%),4분위소득가구비율(%),5분위소득가구비율(%),월_평균_소득_금액,연간_총지출,노인수_x,장애인수,노인수_y
0,종로구,사직동,1.23,9371,7618.699187,9.062665,0.379032,0.0,0.153750,0.245283,...,0.158391,0.142992,0.214959,0.180704,0.302640,5007768.0,223170702000,1808,277,125
1,종로구,삼청동,1.49,2616,1755.704698,10.978350,0.459153,0.0,0.250000,0.832402,...,0.240190,0.211653,0.229489,0.184304,0.133175,3442534.0,11857714000,626,108,45
2,종로구,부암동,2.27,9460,4167.400881,16.725406,0.699515,2.0,0.314583,0.906250,...,0.177386,0.169309,0.195334,0.212384,0.245588,3647449.0,6377388000,1814,318,174
3,종로구,평창동,8.87,17743,2000.338219,65.354340,2.733348,2.0,0.229615,0.974132,...,0.119703,0.139261,0.173669,0.199565,0.367983,4504006.0,9726599000,3495,513,144
4,종로구,무악동,0.36,8045,22347.222220,2.652487,0.110936,0.0,0.346667,1.000000,...,0.157974,0.118292,0.160620,0.181028,0.381708,5136717.0,8331265000,1476,322,206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,강동구,성내2동,0.67,23414,34946.268660,5.893320,0.239663,19.0,0.055000,0.886957,...,0.221124,0.224524,0.233450,0.202741,0.118160,2824592.0,28749704000,4469,1071,930
656,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,...,0.180404,0.192774,0.234538,0.218861,0.173423,2897346.0,27366893000,4098,876,706
657,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,...,0.160445,0.201655,0.231001,0.214678,0.192221,2840603.0,48142577000,8478,1983,1637
658,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,...,0.137338,0.148917,0.193626,0.235011,0.285108,3414406.0,17302683000,4250,935,536


In [ ]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GU_NAME        660 non-null    object 
 1   ADM_NM         660 non-null    object 
 2   동면적(k㎡)        660 non-null    float64
 3   인구수            660 non-null    int64  
 4   인구밀도(명/km²)    660 non-null    float64
 5   동단위우량          660 non-null    float64
 6   동단위시우량         660 non-null    float64
 7   침수횟수           660 non-null    float64
 8   경사도            660 non-null    float64
 9   양호비율           660 non-null    float64
 10  녹지면적           660 non-null    float64
 11  하천거리(km)       660 non-null    float64
 12  빗물펌프장까지의거리(m)  660 non-null    float64
 13  제방             660 non-null    int64  
 14  수문             660 non-null    int64  
 15  암거             660 non-null    int64  
 16  도로_건물_면적       660 non-null    float64
 17  병원             660 non-null    int64  
 18  경찰        

In [ ]:
merge_data.isna().sum()

GU_NAME          0
ADM_NM           0
동면적(k㎡)          0
인구수              0
인구밀도(명/km²)      0
동단위우량            0
동단위시우량           0
침수횟수             0
경사도              0
양호비율             0
녹지면적             0
하천거리(km)         0
빗물펌프장까지의거리(m)    0
제방               0
수문               0
암거               0
도로_건물_면적         0
병원               0
경찰               0
소방서              0
노후건물개수           0
1분위소득가구비율(%)     0
2분위소득가구비율(%)     0
3분위소득가구비율(%)     0
4분위소득가구비율(%)     0
5분위소득가구비율(%)     0
월_평균_소득_금액       0
연간_총지출           0
노인수_x            0
장애인수             0
노인수_y            0
dtype: int64

In [ ]:
test=merge_data.dropna(axis=0)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GU_NAME        660 non-null    object 
 1   ADM_NM         660 non-null    object 
 2   동면적(k㎡)        660 non-null    float64
 3   인구수            660 non-null    int64  
 4   인구밀도(명/km²)    660 non-null    float64
 5   동단위우량          660 non-null    float64
 6   동단위시우량         660 non-null    float64
 7   침수횟수           660 non-null    float64
 8   경사도            660 non-null    float64
 9   양호비율           660 non-null    float64
 10  녹지면적           660 non-null    float64
 11  하천거리(km)       660 non-null    float64
 12  빗물펌프장까지의거리(m)  660 non-null    float64
 13  제방             660 non-null    int64  
 14  수문             660 non-null    int64  
 15  암거             660 non-null    int64  
 16  도로_건물_면적       660 non-null    float64
 17  병원             660 non-null    int64  
 18  경찰        

In [ ]:
test.isna().sum()

GU_NAME          0
ADM_NM           0
동면적(k㎡)          0
인구수              0
인구밀도(명/km²)      0
동단위우량            0
동단위시우량           0
침수횟수             0
경사도              0
양호비율             0
녹지면적             0
하천거리(km)         0
빗물펌프장까지의거리(m)    0
제방               0
수문               0
암거               0
도로_건물_면적         0
병원               0
경찰               0
소방서              0
노후건물개수           0
1분위소득가구비율(%)     0
2분위소득가구비율(%)     0
3분위소득가구비율(%)     0
4분위소득가구비율(%)     0
5분위소득가구비율(%)     0
월_평균_소득_금액       0
연간_총지출           0
노인수_x            0
장애인수             0
노인수_y            0
dtype: int64

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,양호비율,...,1분위소득가구비율(%),2분위소득가구비율(%),3분위소득가구비율(%),4분위소득가구비율(%),5분위소득가구비율(%),월_평균_소득_금액,연간_총지출,노인수_x,장애인수,노인수_y
0,종로구,사직동,1.23,9371,7618.699187,9.062665,0.379032,0.0,0.153750,0.245283,...,0.158391,0.142992,0.214959,0.180704,0.302640,5007768.0,223170702000,1808,277,125
1,종로구,삼청동,1.49,2616,1755.704698,10.978350,0.459153,0.0,0.250000,0.832402,...,0.240190,0.211653,0.229489,0.184304,0.133175,3442534.0,11857714000,626,108,45
2,종로구,부암동,2.27,9460,4167.400881,16.725406,0.699515,2.0,0.314583,0.906250,...,0.177386,0.169309,0.195334,0.212384,0.245588,3647449.0,6377388000,1814,318,174
3,종로구,평창동,8.87,17743,2000.338219,65.354340,2.733348,2.0,0.229615,0.974132,...,0.119703,0.139261,0.173669,0.199565,0.367983,4504006.0,9726599000,3495,513,144
4,종로구,무악동,0.36,8045,22347.222220,2.652487,0.110936,0.0,0.346667,1.000000,...,0.157974,0.118292,0.160620,0.181028,0.381708,5136717.0,8331265000,1476,322,206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,강동구,성내2동,0.67,23414,34946.268660,5.893320,0.239663,19.0,0.055000,0.886957,...,0.221124,0.224524,0.233450,0.202741,0.118160,2824592.0,28749704000,4469,1071,930
656,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12.0,0.072143,0.633929,...,0.180404,0.192774,0.234538,0.218861,0.173423,2897346.0,27366893000,4098,876,706
657,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9.0,0.176154,0.608911,...,0.160445,0.201655,0.231001,0.214678,0.192221,2840603.0,48142577000,8478,1983,1637
658,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10.0,0.170909,0.797872,...,0.137338,0.148917,0.193626,0.235011,0.285108,3414406.0,17302683000,4250,935,536


In [ ]:
merge_data.drop_duplicates(subset=['ADM_NM']).to_excel("./행정동.xlsx")